In [76]:
import pandas as pd
import numpy as np
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [77]:
df_products = pd.read_json(f'../../../data/processed/products.json.gz', orient="records", compression="gzip")

In [78]:
df_products.head()

,keywords,categories
0,sony mdrv500dj monitor series headphone swivel...,[headphones]
1,fujifilm mx2900 23mp digital camera 3x optical...,[cameras]
2,sony mvcfd95 mavica 2mp digital camera 10x opt...,[cameras]
3,plantronics h141 duoset convertible headset di...,[headphones]
4,fujifilm finepix 4900 43mp digital camera 6x o...,[cameras]


In [79]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df=0.99, sublinear_tf=True)
item_feature_matrix = tfidf_vectorizer.fit_transform(df_products['keywords'])
item_similarity_matrix = cosine_similarity(item_feature_matrix)

In [80]:
len(tfidf_vectorizer.get_feature_names_out())

94813

In [81]:
len(item_similarity_matrix)

1798

In [83]:
pickle.dump(tfidf_vectorizer, open('../../../models/content_based_filtering/tfidf_vectorizer.pkl', 'wb'))
pickle.dump(item_feature_matrix, open('../../../models/content_based_filtering/item_feature_matrix.pkl', 'wb'))
pickle.dump(item_similarity_matrix, open('../../../models/content_based_filtering/item_similarity_matrix.pkl', 'wb'))